In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/pSp_self_customize')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import dlib
import glob

import cv2
import numpy as np

from tqdm import tqdm
from PIL import Image

In [3]:
##############################################################################
"""
test_input = "image_datasets/test_images"
test_original_path = "image_datasets/test_resized"
test_processed_path = "image_datasets/test_blurred"

os.makedirs(test_original_path, exist_ok=True)
os.makedirs(test_processed_path, exist_ok=True)
"""
##############################################################################
"""
input_path = "image_datasets/ffhq5k_6k_related/ffhq5k_6k_aligned"
output_ori_path = "image_datasets/ffhq5k_6k_related/ffhq5k_6k_blurred_ksize_045/ffhq5k_6k_blurred_ori"
output_pro_path = "image_datasets/ffhq5k_6k_related/ffhq5k_6k_blurred_ksize_045/ffhq5k_6k_blurred"
"""
##############################################################################
# 349, 065, 059, 043, 039, 033, 031
input_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_aligned"
output_ori_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_349/ffhq10k_11k_blurred_ori"
output_pro_path = "image_datasets/ffhq10k_11k_related/ffhq10k_11k_blurred_ksize_349/ffhq10k_11k_blurred"

os.makedirs(output_ori_path, exist_ok=True)
os.makedirs(output_pro_path, exist_ok=True)


In [4]:
#Load face landmark detection
detector = dlib.get_frontal_face_detector()


In [5]:
def blur(img, face_rectangle, ksize, sigma):
  """
  In this method, instead of a box filter, a Gaussian kernel is used. 
  It is done with the function, cv.GaussianBlur(). 
  We should specify the width and height of the kernel which should be positive and odd. 
  We also should specify the standard deviation in the X and Y directions, sigmaX and sigmaY respectively. 
  If only sigmaX is specified, sigmaY is taken as the same as sigmaX. 
  If both are given as zeros, they are calculated from the kernel size. 
  Gaussian blurring is highly effective in removing Gaussian noise from an image.
  """
  topleft_x = face_rectangle.left()
  topleft_y = face_rectangle.top()
  w = face_rectangle.right() - face_rectangle.left()
  h = face_rectangle.bottom() - face_rectangle.top()

  roi = img[topleft_y: topleft_y+h, topleft_x: topleft_x+w]
  blur = cv2.GaussianBlur(roi, (ksize,ksize), sigmaX=sigma) #  sigmaY default=0 if sigmaY is zero, it is set to be equal to sigmaX.

  img[topleft_y: topleft_y+h, topleft_x: topleft_x+w] = blur

  return img


In [6]:
counts = 0
file_paths = glob.glob(os.path.join(input_path,"*.png"))
try:
  with tqdm(file_paths, total=len(file_paths)) as t:
    for f in t:
      image_np = dlib.load_rgb_image(f)
      dets = detector(image_np,1)

      image_cv = cv2.imread(f)
      try:
        blurred_image = blur(img=image_cv.copy(), face_rectangle=dets[0], ksize=349, sigma=0)
        f_name = f.split("/")[-1]

        image_cv = cv2.resize(image_cv,(256,256))
        blurred_image = cv2.resize(blurred_image,(256,256))
        
        Image.fromarray(cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB)).save(os.path.join(output_ori_path, f_name))
        Image.fromarray(cv2.cvtColor(blurred_image, cv2.COLOR_BGR2RGB)).save(os.path.join(output_pro_path,"blurred_"+f_name))
      except:
        continue
      counts += 1      
except KeyboardInterrupt:
  t.close()
  raise

print("\n{} images have been blurred!".format(counts))


100%|██████████| 998/998 [15:55<00:00,  1.04it/s]


988 images have been blurred!
